In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import tensorflow as tf
import nltk

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install fugashi unidic_lite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unidic_lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658818 sha256=41d666e0ce08f5e97081b69426122db1a69ec9034f244ad96d1f01c9448b5dd4
  Stored in directory: /root/.cache/pip/wheels/89/e8/68/f9ac36b8cc6c8b3c96888cd57434abed96595d444f42243853
Successfully built unidic_lite


In [4]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [5]:
from transformers import AutoTokenizer, AutoModel

# # 문장을 tokenize할 tokenizer와 model을 다운로드
tokenizer = AutoTokenizer.from_pretrained("tohoku-nlp/bert-base-japanese-v3")
# model = AutoModel.from_pretrained("beomi/KcELECTRA-base-v2022")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

In [6]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [7]:
#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [8]:
#GPU 사용
device = torch.device("cuda:0")

In [9]:
import pandas as pd

df=pd.read_csv("/content/drive/MyDrive/jp_bert/bert_study.csv") ##, encoding='utf-16', sep='\t')
df_test = df.loc[:,['反転前文脈','ジャンル']]
df_test.head()

,反転前文脈,ジャンル
0,いてっ持を貝らほくふをそう…ぬた持もを貝。とこの役手相,중학생
1,りわ変と間人のうつふも伏山。んせまきではとこす出を物べ食にうよの品手,중학생
2,ら知もでまくあ、にずせりたし省反りためとみをとこい悪たっやが分自、,중학생
3,はでいいもてっやてせさべ食いらく柿、たま 。すでのるいてっ言,중학생
4,はでのいませが心てし決、は主ち持の柿。んせまりまたはてれらべ食,중학생


In [10]:
max([len(x) for x in df_test['反転前文脈'].values])

62

In [11]:
df_test['ジャンル'].unique()

array(['중학생', '초등학생', '유치원', '고등학생', '일반인'], dtype=object)

In [12]:
df_test.groupby('ジャンル').count()

,反転前文脈
ジャンル,
고등학생,2189
유치원,46
일반인,10972
중학생,868
초등학생,447


In [13]:
label_dict = {}

for idx, label in enumerate(df_test['ジャンル'].unique()):
  label_dict[label] = idx

df_test['ジャンル'] = df_test['ジャンル'].map(label_dict)

In [14]:
df_test.head()

,反転前文脈,ジャンル
0,いてっ持を貝らほくふをそう…ぬた持もを貝。とこの役手相,0
1,りわ変と間人のうつふも伏山。んせまきではとこす出を物べ食にうよの品手,0
2,ら知もでまくあ、にずせりたし省反りためとみをとこい悪たっやが分自、,0
3,はでいいもてっやてせさべ食いらく柿、たま 。すでのるいてっ言,0
4,はでのいませが心てし決、は主ち持の柿。んせまりまたはてれらべ食,0


In [17]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split


data = df_test[['反転前文脈', 'ジャンル']]
dataset_train, dataset_test = train_test_split(data, test_size=0.1, random_state=0)
dataset_train, dataset_valid = train_test_split(dataset_train, test_size=0.1, random_state=0)

# 내용 확인
print("데이터 수 :", len(dataset_train), "\n", dataset_train.iloc[0])
print("데이터 수 :", len(dataset_valid), "\n", dataset_valid.iloc[0])
print("데이터 수 :", len(dataset_test), "\n", dataset_test.iloc[0])

데이터 수 : 11762 
 反転前文脈    つつれさ用採で国各，てしと統系位単るあの性貫一，るすくとっなが
ジャンル                                   4
Name: 6056, dtype: object
데이터 수 : 1307 
 反転前文脈    えいはとるあが用作るす制抑を長成の子種芽発のンコイダカツハ，が質物学化
ジャンル                                       4
Name: 1290, dtype: object
데이터 수 : 1453 
 反転前文脈    きではで院病なさ小や所療診，りたあに療診で代交が師医科歯・師医の
ジャンル                                    4
Name: 2767, dtype: object


In [18]:
# Setting parameters
max_len = 100
batch_size = 32
warmup_ratio = 0.1
num_epochs = 100
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-6

In [19]:
# train_data tokenize

encoded_train = tokenizer(
    dataset_train['反転前文脈'].tolist(),
    return_tensors='pt',
    max_length=max_len,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 확인
print( encoded_train['input_ids'][0] )
print( encoded_train['token_type_ids'][0]  )
print( encoded_train['attention_mask'][0] )
print()
print('디코딩 :',tokenizer.decode(encoded_train['input_ids'][0]))

tensor([    2, 13819,   494,   439,  3908,  2517,   457,  1432,  1216,    27,
          456,   441,  7275,  4593,  4539,   736,  1142,   493, 17277,  2183,
         5768,   601,    27,   493, 29645,  7275,   453,   460,   430,     3,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

디코딩 : [CLS] つつ れ さ 用 採 で 国 各, て しと 統 系 位 単 る あの 性 貫 一, る すくと っ な が [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [20]:
# valid data tokenized

encoded_valid = tokenizer(
    dataset_valid['反転前文脈'].tolist(),
    return_tensors='pt',
    max_length=max_len,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

print('디코딩 :',tokenizer.decode(encoded_valid['input_ids'][0]))
print('디코딩 :',tokenizer.decode(encoded_valid['input_ids'][-1]))

디코딩 : [CLS] え いは とる あ が 用作 るす 制 抑 を 長成 の 子種 芽 発 の ンコイダカツハ, が 質物 学 化 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
디코딩 : [CLS] て 見 を ん さ 父 お くず なう も 度 何 に うそ しれ う 、 と 」 。 か うそ か [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


In [21]:
from torch.utils.data import Dataset, DataLoader

# 데이터셋 클래스
class ReviewDataset(Dataset):
#데이터셋 정의(초기화) (A)
  def __init__(self, data, labels):
    self.data = data
    self.labels = labels

#데이터셋 길이(B)
  def __len__(self):
    return len(self.labels)

#출력 (C)
  def __getitem__(self, index):
    item = {'input_ids': self.data['input_ids'][index],
            'token_type_ids': self.data['token_type_ids'][index],
            'attention_mask': self.data['attention_mask'][index]}
    item['labels'] = torch.tensor(self.labels[index])
    return item
# 데이터셋 생성
train_dataset = ReviewDataset(encoded_train, dataset_train['ジャンル'].values)
valid_dataset = ReviewDataset(encoded_valid, dataset_valid['ジャンル'].values)


len(valid_dataset)

1307

In [22]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

model = AutoModelForSequenceClassification.from_pretrained('tohoku-nlp/bert-base-japanese-v3', num_labels=5)
model.to(device)

config.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/447M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at tohoku-nlp/bert-base-japanese-v3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32768, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [23]:
training_args = TrainingArguments(
    output_dir = './outputs', # model이 저장되는 directory
    logging_dir = './logs', # log가 저장되는 directory
    num_train_epochs = 10, # training epoch 수
    per_device_train_batch_size=32,  # train batch size
    per_device_eval_batch_size=32,   # eval batch size
    logging_steps = 200, # logging step, batch단위로 학습하기 때문에 epoch수를 곱한 전체 데이터 크기를 batch크기로 나누면 총 step 갯수를 알 수 있다.
    save_steps= 50, # 50 step마다 모델을 저장한다.
    evaluation_strategy="steps",
    save_total_limit=2 # 2개 모델만 저장한다.
)

#정확도 측정을 위한 함수 정의
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # average: 'micro', 'macro', 'weighted' or 'samples'
    # 참고 https://aimb.tistory.com/152
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'acc': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

Step,Training Loss,Validation Loss,Acc,F1,Precision,Recall
200,0.671200,0.601062,0.757460,0.765311,0.777714,0.757460
400,0.528500,0.561300,0.769702,0.784942,0.810616,0.769702
600,0.410000,0.460976,0.815608,0.819442,0.831483,0.815608
800,0.345900,0.497484,0.834736,0.839327,0.851726,0.834736
1000,0.239100,0.565064,0.854629,0.839259,0.838889,0.854629
1200,0.187700,0.608499,0.854629,0.843706,0.844315,0.854629
1400,0.139700,0.549602,0.851568,0.841079,0.837299,0.851568
1600,0.100500,0.662131,0.843917,0.845534,0.848435,0.843917
1800,0.079400,0.670389,0.859985,0.856988,0.857244,0.859985
2000,0.052300,0.864334,0.843152,0.849329,0.859431,0.843152


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

TrainOutput(global_step=3680, training_loss=0.15680524499520013, metrics={'train_runtime': 3356.765, 'train_samples_per_second': 35.04, 'train_steps_per_second': 1.096, 'total_flos': 3626713583978400.0, 'train_loss': 0.15680524499520013, 'epoch': 10.0})

In [27]:
trainer.evaluate(valid_dataset)

{'eval_loss': 0.944762647151947,
 'eval_acc': 0.856159143075746,
 'eval_f1': 0.8559581180850914,
 'eval_precision': 0.8559553968658788,
 'eval_recall': 0.856159143075746,
 'eval_runtime': 3.4994,
 'eval_samples_per_second': 373.493,
 'eval_steps_per_second': 11.716,
 'epoch': 10.0}

In [ ]:
# df=pd.read_csv("/content/drive/MyDrive/jp_bert/bert_study.csv") ##, encoding='utf-16', sep='\t')
# df_test = df.loc[:,['反転前文脈','ジャンル']]
# df_test.head()

,反転前文脈,ジャンル
0,いてっ持を貝らほくふをそう…ぬた持もを貝。とこの役手相,중학생
1,りわ変と間人のうつふも伏山。んせまきではとこす出を物べ食にうよの品手,중학생
2,ら知もでまくあ、にずせりたし省反りためとみをとこい悪たっやが分自、,중학생
3,はでいいもてっやてせさべ食いらく柿、たま 。すでのるいてっ言,중학생
4,はでのいませが心てし決、は主ち持の柿。んせまりまたはてれらべ食,중학생


In [28]:
# train_data tokenize

encoded_test = tokenizer(
    dataset_test['反転前文脈'].tolist(),
    return_tensors='pt',
    max_length=max_len,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 확인
print( encoded_test['input_ids'][0] )
print( encoded_test['token_type_ids'][0]  )
print( encoded_test['attention_mask'][0] )
print()
print('디코딩 :',tokenizer.decode(encoded_test['input_ids'][0]))

tensor([    2,   431,   457,   465,   457,  6410,  3970, 14770,  1829,   486,
         2386,  4011,  9398,    27,   492,   449,  7364,   461,  4011,  9398,
          457,   700,   670,   430,  1974, 20041,  3180,   593,  1974,  1124,
          464,     3,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

디코딩 : [CLS] き で は で 院 病 なさ 小 や 所 療診, り たあ に 療診 で 代 交 が 師 医科 歯 ・ 師 医 の [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


In [32]:
zero_labels = np.zeros(len(dataset_test), dtype=int)
test_dataset = ReviewDataset(encoded_test, zero_labels)

In [41]:
predictions = trainer.predict(test_dataset)
print(predictions.predictions.shape, predictions.label_ids.shape)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(1453, 5) (1453,)


In [42]:
print(label_dict)

{'중학생': 0, '초등학생': 1, '유치원': 2, '고등학생': 3, '일반인': 4}


In [43]:
reverse_dict = {
    0:'중학생',
    1:'초등학생',
    2:'유치원',
    3:'고등학생',
    4:'일반인',
}

In [44]:
dataset_test['pred_target'] = np.array([ np.argmax(p) for p in predictions.predictions ], dtype='int8')

for text, label, target in zip(dataset_test['反転前文脈'], dataset_test['pred_target'], dataset_test['ジャンル']):

  print(f'input : {text}')
  print('----'*50)
  print(f'>> 해당 문장은 {reverse_dict[label]} 입니다. 실제 정답 {reverse_dict[target]}')
  print()

input : きではで院病なさ小や所療診，りたあに療診で代交が師医科歯・師医の
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
>> 해당 문장은 일반인 입니다. 실제 정답 일반인

input : 　いぱっい，あわ「。よたし　うふく　をのもれ入　もしたわ。よた
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
>> 해당 문장은 중학생 입니다. 실제 정답 유치원

input : しは焼燃もてし熱で中気空，は銅，たま。るなにムウシネグマ化酸，
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
>> 해당 문장은 고등학생 입니다. 실제 정답 고등학생

input : てしうどは服衣たっなくな着　イア活生例の法方納収の服衣　図
--------------------------------------------------------------------------------------------------------------------------------------------------